In [ ]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd

In [ ]:
#Fonction qui sera appelée pour chaque éléments récupérés
def transform(x):
    #Si c'est une POS, on retourne la valeur textuelle de la balise
    if x.attrs["class"][0] == "tag_wordtype": return x.text[0] + "."
    #Si c'est un word, on retourne la valeur textuelle de la balise
    if x.attrs["class"][0] == "dictLink": return x.text
    #Si c'est un audio, on va récupérer toutes les valeurs dans un dict
    if x.attrs["class"][0] == "audio": 
        #On créé un dictionnaire qui contiendra en clé la langue et en valeur le lien mp3
        dictResult = {}
        #Traitement textuel pour récupérer le lien du mp3
        value = x.attrs["onclick"].split(",")[1:]
        value[-1] = value[-1][:-2]
        for i in range(0, len(value), 2):
            dictResult[value[i+1][1:-1]] = "https://www.linguee.fr/mp3/" + value[i][1:-1] + ".mp3"
        return dictResult

In [ ]:
def getWord(word):
    listAllWords = []
    #On effectue la requête sur le site
    response = requests.get("https://www.linguee.fr/anglais-francais/traduction/" + word + ".html") 
    soup = BeautifulSoup(response.text,'html.parser')
    soupListh2 = soup.findAll('h2',{"class":"lemma_desc"})
    #Pour tous les élements h2   
    for h2 in soupListh2:
        #On récupère tous les éléments de type lien et du wordType
        link = h2.findAll("a")
        pos = h2.findAll('span', {"class": "tag_wordtype"})
        link = link + pos
        if len(link) >= 3:
            #On applique la transformation pour récupèrer les valeurs propres
            listWord = list(map(lambda x: transform(x), link))
            listAllWords.append([i for i in listWord if i])
    #On retourne un dataframe
    return pd.DataFrame(listAllWords, columns =['Word', 'Sound', 'Pos'])

In [ ]:
getWord("beat")

,Word,Sound,Pos
0,beat,{'anglais américain': 'https://www.linguee.fr/...,v.
1,beat,{'anglais américain': 'https://www.linguee.fr/...,n.
2,beat,{'anglais américain': 'https://www.linguee.fr/...,a.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=28a13d76-245b-4e36-8dd1-db1382e81cfa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>